## Failte Ireland API

In [ ]:
pip install requests

In [ ]:
import requests, json

In [ ]:
# Connect to Data.Gov Activities API and print response
response = requests.get("https://failteireland.portal.azure-api.net/docs/services/opendata-api-v1/operations/activities-get")
print(response.status_code)
print(response)

In [ ]:
# Review if data we received back from the API is JSON:
url = "https://failteireland.portal.azure-api.net/docs/services/opendata-api-v1/operations/activities-get"
r = requests.get(url)

if 'json' in r.headers.get('Content-Type'):
    js = r.json()
else:
    print('Response content is not in JSON format.')
    js = 'spam'

In [ ]:
#Read non JSON API data using URL Parse 
import http.client, urllib.request, urllib.parse, urllib.error, base64

import pandas as pd

headers = {}

params = urllib.parse.urlencode({})
jsondata = []
try:
    conn = http.client.HTTPSConnection('failteireland.azure-api.net')
    conn.request("GET", "/opendata-api/v1/activities?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
#    for r in data:
#        print(r)
    jsond = json.loads(data)
    jsondata.append(jsond)
    print(jsondata)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))


In [ ]:
# Reattempt to read API Data with dataframe

import urllib.request as request
import pandas as pd
import json
Name, Telephone = [],[]
jsondata = []

with request.urlopen('https://failteireland.azure-api.net/opendata-api/v1/activities') as response:
    source = response.read()
for line in source:
    try:
        jsond = json.loads(source)
        jsond2 = jsond['results']
        for i in jsondata:
            Name.append(i['name'])
            Telephone.append(i['telephone'])
        df = pd.DataFrame([Name,Telephone]).T
#        jsondata.append(jsond)
    except Exception as e:
        print(e)            
#    jsondata = json.loads(source)
#    jsond = json.loads(source)
#    jsondata.append(jsond)

In [ ]:
# Panda Flatten Script
from itertools import chain, starmap

def flatten_json_iterative_solution(dictionary):
    """Flatten a nested json file"""

    def unpack(parent_key, parent_value):
        """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                temp1 = parent_key + '_' + key
                yield temp1, value
        elif isinstance(parent_value, list):
            i = 0 
            for value in parent_value:
                temp2 = parent_key + '_'+str(i) 
                i += 1
                yield temp2, value
        else:
            yield parent_key, parent_value    

            
    # Keep iterating until the termination condition is satisfied
    while True:
        # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
        dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
        # Terminate condition: not any value in the json file is dictionary or list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary

### Did not need to use this in the end 

In [ ]:
# Dataframe for API
activities = pd.DataFrame()
activities['name'] = list(map(lambda jsond: jsond['name'], jsondata))
activities['url'] = list(map(lambda jsond: jsond['url'], jsondata))
activities['AddressRegion'] = list(map(lambda jsond: jsond['AddressRegion'], jsondata))
activities['AddressLocality'] = list(map(lambda jsond: jsond['AddressLocality'], jsondata))
activities.head()

### Did not need to use this in the end as I exported to CSV as shown below

## Exporting to CSV

In [ ]:
# Python program to convert JSON file to CSV
  
import json
import csv
  
# Opening JSON file and loading the data
# into the variable data
with open('activities.json') as json_file:
    data = json.load(json_file)

activities_data = data['results']
  
# now we will open a file for writing
data_file = open('activities_file.csv', 'w')
  
# create the csv writer object
csv_writer = csv.writer(data_file)
  
# Counter variable used for writing 
# headers to the CSV file
count = 0
  
for results in activities_data:
    if count == 0:
  
        # Writing headers of CSV file
        header = results.keys()
        csv_writer.writerow(header)
        count += 1
  
    # Writing data of CSV file
    csv_writer.writerow(results.values())

data_file.close()

In [ ]:
# Reading activities_file dataset 
import pandas as pd
import numpy as np

act_data = pd.read_csv(r"C:\Users\korpe\Desktop\Work\Activities\Activities_Failte.csv")
#print(act_data)

#Dealing with headers
print(act_data.head(5))

In [ ]:
# Saving the csv into a dataFrame
activitiesfile = "C://Users/korpe/Desktop/Work/Activities/Activities_Failte.csv"
activities_df = pd.read_csv(activitiesfile)
print(activities_df)

# View the head of the DataFrame
print(activities_df.head())

In [ ]:
# Using Pandas, create an array from the dataframe
activities_df_array = activities_df.values
print(activities_df.head())

# Check the datatype of activities_df_array
print(type(activities_df_array))

In [ ]:
# Print information of 'activities_df'
activities_df.info()

# Before handling missing values in dataframe, will import to Mongodb to save data

## Import into MongoDB

In [ ]:
# Saving pandas dataframe into json before sending to MongoDB
import numpy as np
import pandas as pd
  
dataFrame = activities_df
dataFrame.to_json(r'C:\Users\korpe\Desktop\Work\Activities\Activities_Failte.json')


In [ ]:
# Read json into mongdb
import json
import pymongo
import pandas as pd
from pymongo import MongoClient 
  
# Making Connection
myclient = pymongo.MongoClient('192.168.56.30', 27017)
   
# database 
db = client.activitiesdb
   
# Created or Switched to collection names
Collection = db.activitiescollection
  
# Loading or Opening the dataframe
with open('Activities_Failte.json') as file:
    file_data = json.load(file)
      
# Inserting the loaded data in the Collection if JSON contains data more than one entry insert_many is used else inser_one is used
if isinstance(file_data, list):
    Collection.insert_many(file_data)  
else:
    Collection.insert_one(file_data) 
    
# Print Output of activities db to confirm Mongodb load
for m in Collection.find({}):
 print(m)

# Same procedure used to upload into a new Collection after data wrangling step below.

## Data Wrangling 

In [ ]:
# Using Numpy, create an array from the dataframe
activities_df_array = activities_df.values
print(activities_df.head())

# Check the datatype of activities_df_array
print(type(activities_df_array))

# Print information of 'activities_df'
activities_df.info()

In [244]:
# There are a total of 5413 rows and 9 columns in the dataframe.

# Before handling missing data, create copy of activities_df
activities_df_missing = activities_df.copy()
df = activities_df_missing
#activities_df.head()

# For the merge-join of weather data for the project, the critical data is the Address region. 
# We will need to replace NaN in the AddressRegion column with correct region data.

# Identify missing values in the AddressRegion column
print (df[df['AddressRegion'].isna()])

# Select all rows with NaN under AddressRegion DataFrame column
nan_values = df[df['AddressRegion'].isna()]
print (nan_values)

# Confirm all empty fields using iloc function beginning with the First

df.iloc[379][5]

# Replace NaN values with a Region label based on the locality and URL

df.iat[379,5] = 'Cork'
df.iat[2186,5] = 'Cork'
df.iat[2191,5] = 'Donegal'
df.iat[2298,5] = 'Cork'
df.iat[2358,5] = 'Offaly'
df.iat[2908,5] = 'Wicklow'
df.iat[3167,5] = 'Mayo'
df.iat[3682,5] = 'Cork'
df.iat[3959,5] = 'Galway'
df.iat[3993,5] = 'Cork'

# Confirm No NaN are present in the AddressRegion Column

nan_values = df[df['AddressRegion'].isna()]
print (nan_values)

# The remaning NaN are acceptable as is some cases there are no fields present for these. 

Empty DataFrame
Columns: [Name, Url, Telephone, Longitude, Latitude, AddressRegion, AddressLocality, AddressCountry, Tags]
Index: []


## Miscellanous

In [ ]:
#Instructions for Pandas Dataframe
def flatten_dict(d):
    """ Returns list of lists from given dictionary """
    l = []
    for k, v in sorted(d.items()):
        if isinstance(v, dict):
            flatten_v = flatten_dict(v)
            for my_l in reversed(flatten_v):
                my_l.insert(0, k)

            l.extend(flatten_v)

        elif isinstance(v, list):
            for l_val in v:
                l.append([k, l_val])

        else:
            l.append([k, v])

    return l

df = pd.DataFrame(flatten_dict(jsond))
print(df)

In [ ]:
df = pd.DataFrame(flatten_dict(jsond))
print(df)

In [ ]:
#type(jsondata)
json_string = json.dumps(jsondata)
print(json_string)